In [ ]:
using LinearAlgebra
using BoundaryValueDiffEq
using SparseArrays
using CairoMakie
using ModelingToolkit
using MethodOfLines
using DifferentialEquations
using NonlinearSolve
using DomainSets
using FFTW
using GLMakie

In [ ]:
gamma::Float64 = 0.0
gamma_3::Float64 = 0.0
c::Float64 = 3

omega::Float64 = 2*pi*7

@parameters x,t
@variables A(..), B(..)
r1 = @rule cos(~x)^3 => 0.75 * cos(~x) + 0.25 * cos(3 * ~x)
r2 = @rule sin(~x)^3 => 0.75 * sin(~x) - 0.25 * sin(3 * ~x)
r3 = @rule cos(~x)^2 => 1 - sin(~x)^2
r4 = @rule sin(~x)^2 => 1 - cos(~x)^2


xleft::Float64 = 0.0
xright::Float64 = 1.0
N = 1000
order = 2
stepx = (xright-xleft)/N
u = A(x) * sin(omega*t) + B(x) * cos(omega*t)
Dx = Differential(x)
Dt = Differential(t)
y = Dt(Dt(u)) - c*c*Dx(Dx(u)) + gamma*Dt(u) + gamma_3*Dt(u)*Dt(u)*Dt(u)
y_exp = expand_derivatives(y)

y_exp = simplify(expand(y_exp), RuleSet([r1, r2, r3, r4]))
y_exp = expand(y_exp)
y_exp = simplify(y_exp, RuleSet([r1, r2, r3, r4]))

sin_coeff = -Symbolics.coeff(y_exp, sin(omega*t))
cos_coeff = -Symbolics.coeff(y_exp, cos(omega*t))

#=sin_sol = Symbolics.solve_for(sin_coeff ~ 0, Differential(x)(Differential(x)(A(x))))
println("Sin:")
println("Sin coeff: ", sin_coeff)
println("Axx = ", sin_sol)

fA = Symbolics.build_function(sin_sol, A(x), B(x), gamma, omega, c; expression=Val{false})

cos_sol = Symbolics.solve_for(cos_coeff ~ 0, Differential(x)(Differential(x)(B(x))))
println("Cos:")
println("Cos coeff: ", cos_coeff)
println("Bxx = ", cos_sol)

fB = Symbolics.build_function(cos_sol, A(x), B(x), gamma, omega, c; expression=Val{false})=#
     


In [ ]:
function transform_sym(expr_str)
    corrected = string(expr_str)

    repl_lapl = [
        "Differential(x)(Differential(x)(A(x)))" => "(A[i+1] - 2A[i] + A[i-1]) / dx^2",
        "Differential(x)(Differential(x)(B(x)))" => "(B[i+1] - 2B[i] + B[i-1]) / dx^2",
        ]
    repl_first_der = [
        "Differential(x)(A(x))" => "(A[i+1] - A[i-1]) / (2*dx)",
        "Differential(x)(B(x))" => "(B[i+1] - B[i-1]) / (2*dx)",
        ]
    repl_linear = [
        "A(x)" => "A[i]",
        "B(x)" => "B[i]"
        ]

    corrected = replace(corrected, repl_lapl...)
    corrected = replace(corrected, repl_first_der...)
    corrected = replace(corrected, repl_linear...)

    repl_powers = [
        "A[i]^" => "(A[i])^",
        "B[i]^" => "(B[i])^"
        ]
    repl_mult_pattern = r"(?<=\d)(?=[A-Za-z\(])"

    corrected = replace(corrected, repl_powers...)
    corrected = replace(corrected, repl_mult_pattern => "*")

    #Finar corrections (required for scope):
    repl_sym = [
        "A" => "A_array",
        "B" => "B_array",
        "i" => "i_local",
        ]

    corrected = replace(corrected, repl_sym...)

    return corrected
end

In [ ]:
println("Trial substitution:")
println(transform_sym(sin_coeff))

In [ ]:
function create_residual_function_1D(N, sin_eq_str, cos_expr_str)
    function_code = """
    function residual!(F, U, p)
        dx = p

        A_array = @view(U[1:$N+1])
        B_array = @view(U[$N+2:end])
        F_A = @view(F[1:$N+1])
        F_B = @view(F[$N+2:end])

        # Inner points:
        for i_local in 2:$(N)
            F_A[i_local] = ($sin_eq_str) - 100 * exp(-40(i_local * dx)^2)
            F_B[i_local] = ($cos_expr_str) 
        end

        # Dirichlet BCs
        F_A[1] = A_array[1]
        F_A[end] = A_array[end]
        F_B[1] = B_array[1]
        F_B[end] = B_array[end]

        return F
    end
    """
    return eval(Meta.parse(function_code))
end


In [ ]:
sin_eq_str = transform_sym(sin_coeff)
cos_eq_str = transform_sym(cos_coeff)

nonlinear_residual = create_residual_function_1D(N, sin_eq_str, cos_eq_str)
initial_guess = ones(2*(N+1))

prob = NonlinearProblem(nonlinear_residual, initial_guess, stepx)
sol = NonlinearSolve.solve(prob, NewtonRaphson(), reltol = 1e-5, abstol = 1e-5)

In [ ]:
xgrid = collect(range(start=0.0, stop=1.0, step=stepx))
myA  = sol.u[1:N+1]
myB  = sol.u[N+2:end]
# p1 = plot(xgrid, myA, xlabel="x", ylabel = "A(x)")
# p2 = plot(xgrid, myB, xlabel="x", ylabel = "B(x)")
# plot(p1,p2,layout=(2,1))

In [ ]:
Nt = 5
dt = 0.01

# Observable for u(x,t)
u = Observable(myA .* sin(0.0) .+ myB .* cos(0.0))

# Create figure and axis
fig = Figure(resolution = (800, 500))
ax = Axis(fig[1, 1],
    title = "t = 0.0",
    xlabel = "x",
    ylabel = "u(x, t)",
    limits = (nothing, nothing, -1.5, 1.5)
)

# Plot linked to observable
lines!(ax, xgrid, u)

# Display the figure
display(fig)

# Animate it live
@async begin
    for k in 1:Int(Nt/dt)
        t = k * dt
        u[] = myA .* sin(omega * t) .+ myB .* cos(omega * t)
        ax.title = "t = $(round(t, digits=2))"
        sleep(1/60)  # control playback speed (60 FPS)
    end
end


In [ ]:
k = 58
dt = 0.01

fs = N

new_u = myA * sin(omega * (k * dt)) .+ myB * cos(omega * (k * dt))

F = fftshift(fft(new_u))
freqs =  fftshift(fftfreq(length(new_u), fs))

# Plotting with Makie
f = Figure()
ax1 = Axis(f[1, 1],
    xlabel = "Distance (m)",
    ylabel = "Displacement (m)",
)

ax2 = Axis(f[1, 2], xlabel = "Frequency (Hz)",
    ylabel = "Amplitude (m)",
)

# plot the lines
lines!(ax1, xgrid, new_u)
lines!(ax2, freqs, abs.(F) / fs)


# update the layout
xlims!(ax2, -10, 10)

f